# RAG System for Survey Variable Resolution

This Jupyter Notebook is designed to implement a RAG (Retrieval-Augmented Generation) system. The primary purpose of this system is to resolve user-submitted survey variable data and find other variables that are likely to match.

## Overview

The RAG system leverages the power of retrieval-based and generative models for machine learning. It uses a two-step process:

1. **Retrieval**: The system retrieves relevant documents (in this case, survey variables) from a Pinecone DB knowledge source based on the user-submitted data.

2. **Generation**: The system then uses the retrieved documents to generate a response using a Cohere Reranking LLM and OpenAI's ChatOpenAI GPT 3.5 model. This response includes other variables that have a high likelihood of matching the user-submitted data.

## Usage

To use this notebook, input your survey variable data using the `input_data` directory. The RAG system will process this data, retrieve relevant variables from the knowledge source, and generate a list of variables that are likely to match your input.

## Benefits

The RAG system provides a powerful tool for survey data analysis. It can help identify patterns and correlations in the data, which can be invaluable for research and decision-making.

Please note that the accuracy of the system's output depends on the quality and comprehensiveness of the knowledge source. Therefore, it's crucial to continually update and expand the knowledge source to improve the system's performance.

## Setup

In [1]:
import os
import json
import pandas as pd
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever

import cohere
from pinecone import Pinecone
from pinecone import ServerlessSpec



from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

#### API KEYS

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass("OpenAI API Key: ")
COHERE_API_KEY = os.getenv('COHERE_API_KEY') or getpass("COHERE API Key: ")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or getpass("Enter your Pinecone API key: ")



#### Configuration Values

In [57]:

""""
--- MODEL CONFIG ---
"""

PINECONE_INDEX = "biolincc-labels-001" # Name of the Pinecone index
TEMPERATURE = 0
VECTOR_TEXT_FIELD = "text"
EMBEDDING_MODEL = "text-embedding-ada-002"

""""
--- USER SUBMITTED VARIABLE CONFIG ---
"""

USER_SUBMITTED_VARIABLE_COL = "name"
USER_SUBMITTED_LABEL_COL = "description"

#### Logging

In [5]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

### Embedding and Vector DB Setup

In [6]:
pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [7]:
import time

index_name = PINECONE_INDEX
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
  raise Exception("Pinecone index does not exist")

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 259880}},
 'total_vector_count': 259880}

In [8]:
embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

In [9]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone(index, embed.embed_query, VECTOR_TEXT_FIELD)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Parse User Input

In [48]:
# Specify the path to your CSV file
csv_file_path = 'input_data/test_data_dictionary.csv'

# Read the CSV file and convert it into a DataFrame
df = pd.read_csv(csv_file_path)

In [49]:
df.describe()

,Sample/Subject Count
count,336.000000
mean,7418.211310
std,3661.967493
min,12.000000
25%,4388.000000
50%,10122.000000
75%,10370.000000
max,12240.000000


In [50]:
df = df.head(5)
# The column we need to input into the RAG is 'description'
# While in test mode this is all we want to do

In [51]:
user_variables = df['description']

## Setup Retriever


**Generation Prompt**
```
    You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
    You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    For each new variable, a vector search engine has returned the three nearest existing variables found in the data repository. 
    Your job is determine which of these existing variables, if any, is the best fit for harmonizing the new variable to the existing variable given your understanding of the underlying scientific principles.
    You must rationalize why the selected existing variable was chosen over the others, provide context to the relevancy of each existing variable to the new variable.
    Then you are to provide the user with as much information as we can on how they can align their new variable with the selected existing variable.
    
    When there is no obvious match provide additional context for why you can't make a determination.
    
    Contexts:
    {contexts}

    Original question: {query}
```

In [10]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [11]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

output_parser = LineListOutputParser()


### Prompt Templates

#### Output Generation Prompt

In [12]:
# Define the prompt template
template = """
    You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
    You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    For each new variable, a vector search engine has returned the three nearest existing variables found in the data repository. 
    Your job is determine which of these existing variables, if any, is the best fit for harmonizing the new variable to the existing variable given your understanding of the underlying scientific principles.
    You must rationalize why the selected existing variable was chosen over the others, provide context to the relevancy of each existing variable to the new variable.
    Then you are to provide the user with as much information as we can on how they can align their new variable with the selected existing variable.
    
    When there is no obvious match provide additional context for why you can't make a determination.
    
    Contexts:
    {contexts}

    Original question: {query}
"""

GEN_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template=template,
)
llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# Chain
gen_chain = LLMChain(llm=llm, prompt=GEN_PROMPT, output_parser=output_parser)

#### Query Prompt Placeholder

In [63]:
### IS THIS NEEDED? TBD
# query_template = """
#     You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
#     You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    
#     Original query: {query}
# """

# QUERY_PROMPT = PromptTemplate(
#     # input_variables=["question"],
#     input_variables=["query"],
#     template=query_template,
# )
# llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# # Chain
# llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

#### Cohere Rerank Retriever

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere

In [14]:
cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY)
retriever=ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=vectorstore.as_retriever() 
)


## Connecting RAG elements

In [15]:
from langchain.chains import TransformChain
from langchain.chains import SequentialChain

In [34]:
def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    metadata = pd.DataFrame([d.metadata for d in docs], columns=['uid', 'relevance_score'])
    docs = [d.page_content for d in docs]
    print(docs)
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs),
        "metadata": metadata
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts", "metadata"],
    transform=retrieval_transform
)

In [35]:

rag_chain = SequentialChain(
    chains=[retrieval_chain, gen_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text", "metadata"],
)

### Execute the Model

In [61]:
def format_mapped_vars(out, user_var):

    return pd.DataFrame(
        {
            "user_variable": user_var,
            "user_description": out["query"],
            "uid": out["metadata"]["uid"],
            "relevance_score": out["metadata"]["relevance_score"],
        }
    )

In [62]:
def format_llm_res_df(out, user_var):

    return pd.DataFrame(
        {
            "user_variable": user_var,
            "user_description": out["query"],
            "llm_response": out["text"]
        }
    )

In [69]:
def execute_rag_chain(df, mapped_vars_df, llm_res_df):
    for index, row in df.iterrows():
        question = row[USER_SUBMITTED_LABEL_COL]
        print(question)
        out = rag_chain({"question": question})
        mapped_vars_df = pd.concat([mapped_vars_df, format_mapped_vars(out, row[USER_SUBMITTED_VARIABLE_COL])])
        llm_res_df = pd.concat([llm_res_df, format_llm_res_df(out, row[USER_SUBMITTED_VARIABLE_COL])])
    return mapped_vars_df, llm_res_df

## Map to orginal data

In [70]:
mapped_vars_df = pd.DataFrame(columns=[
    "user_variable", # lowest
    "user_description", # out[query] i.e. middle
    "uid", # highest out[metadata][uid]
    "relevance_score",  # highest out[metadata][uid]
])


In [71]:
llm_res_df = pd.DataFrame(columns=[
    "user_variable", # lowest
    "user_description", # out[query] i.e. middle
    "llm_response", # out[text] i.e. middle
])


In [73]:
mapped_vars_df, llm_res_df = execute_rag_chain(df, mapped_vars_df, llm_res_df)

Consent group as determined by DAC
['CONSENT RESEARCH SAMPLE', 'CONSENT STATUS', 'CONSENT SIGNED BY PARTICIPANT']


/var/folders/mn/lqj94hqs2js64_82jtssv38h0000gn/T/ipykernel_34716/102934794.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mapped_vars_df = pd.concat([mapped_vars_df, format_mapped_vars(out, row[USER_SUBMITTED_VARIABLE_COL])])


Source repository where subjects originate
['SOURCE FILE', 'SOURCE', 'SUBJECT LOCATION']
Subject ID used in the Source Repository
['SUBJECT_ID', 'SUBJECT ID', 'UNIQUE SUBJECT ID']
Affection status
['AFFECTIONATE - SCORE', 'AFFECTIONATE SUPPORT (0-100)', 'BASELINE ESSA3:LOVE, AFFECTION AVAILABLE']
Source repository where samples originate
['HAVE ORIGINAL SAMPLES FOR TESTING', 'SOURCE', 'SOURCE FILE']


In [83]:
mapped_vars_df.to_csv('llm_output/mapped_vars_df.csv', index=False)

mapped_vars_df.head()


,user_variable,user_description,uid,relevance_score
0,CONSENT,Consent group as determined by DAC,52783.0,0.454563
1,CONSENT,Consent group as determined by DAC,52786.0,0.124745
2,CONSENT,Consent group as determined by DAC,52785.0,0.066448
0,SUBJECT_SOURCE,Source repository where subjects originate,216548.0,0.084342
1,SUBJECT_SOURCE,Source repository where subjects originate,216503.0,0.077100


In [84]:
llm_res_df.to_csv('llm_output/llm_res_df.csv', index=False)


llm_res_df.head()

,user_variable,user_description,llm_response
0,CONSENT,Consent group as determined by DAC,"(lines, [Based on the provided existing variab..."
0,SUBJECT_SOURCE,Source repository where subjects originate,"(lines, [Based on the context provided, the ne..."
0,SOURCE_SUBJECT_ID,Subject ID used in the Source Repository,"(lines, [Based on the provided contexts, the b..."
0,AFFECTION_STATUS,Affection status,"(lines, [Based on the provided existing variab..."
0,SAMPLE_SOURCE,Source repository where samples originate,"(lines, [Nearest existing variables found in t..."
